In [2]:
import argparse
import json
import sqlite3
import time
from datetime import datetime, timezone
from typing import Any, Dict, Iterable, Optional

import requests


DDL = """
PRAGMA journal_mode=WAL;

CREATE TABLE IF NOT EXISTS ticks (
  id                INTEGER PRIMARY KEY AUTOINCREMENT,
  fetched_at_utc    TEXT    NOT NULL,
  source_ts_ms      INTEGER,
  source_date_text  TEXT,
  currency          TEXT    NOT NULL,

  xau_price         REAL,
  xag_price         REAL,
  chg_xau           REAL,
  chg_xag           REAL,
  pc_xau            REAL,
  pc_xag            REAL,
  xau_close         REAL,
  xag_close         REAL,

  raw_json          TEXT    NOT NULL,

  UNIQUE(currency, source_ts_ms)
);

CREATE INDEX IF NOT EXISTS idx_ticks_fetched_at ON ticks(fetched_at_utc);
CREATE INDEX IF NOT EXISTS idx_ticks_currency   ON ticks(currency);
"""


def now_utc_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="seconds")


def get_json(url: str, timeout_s: int = 15) -> Dict[str, Any]:
    headers = {
        "User-Agent": "Mozilla/5.0 (compatible; goldprice-to-sqlite/1.0; +https://example.invalid)"
    }
    r = requests.get(url, headers=headers, timeout=timeout_s)
    r.raise_for_status()
    return r.json()


def extract_item(payload: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    items = payload.get("items")
    if not isinstance(items, list) or not items:
        return None
    item = items[0]
    return item if isinstance(item, dict) else None


def init_db(conn: sqlite3.Connection) -> None:
    conn.executescript(DDL)
    conn.commit()


def insert_tick(conn: sqlite3.Connection, currency: str, payload: Dict[str, Any]) -> None:
    item = extract_item(payload)
    if item is None:
        raise ValueError("Onverwacht JSON-formaat: geen items[0] gevonden")

    row = {
        "fetched_at_utc": now_utc_iso(),
        "source_ts_ms": payload.get("ts"),
        "source_date_text": payload.get("date"),
        "currency": item.get("curr") or currency,

        "xau_price": item.get("xauPrice"),
        "xag_price": item.get("xagPrice"),
        "chg_xau": item.get("chgXau"),
        "chg_xag": item.get("chgXag"),
        "pc_xau": item.get("pcXau"),
        "pc_xag": item.get("pcXag"),
        "xau_close": item.get("xauClose"),
        "xag_close": item.get("xagClose"),

        "raw_json": json.dumps(payload, ensure_ascii=False, separators=(",", ":")),
    }

    sql = """
    INSERT OR IGNORE INTO ticks (
      fetched_at_utc, source_ts_ms, source_date_text, currency,
      xau_price, xag_price, chg_xau, chg_xag, pc_xau, pc_xag, xau_close, xag_close,
      raw_json
    ) VALUES (
      :fetched_at_utc, :source_ts_ms, :source_date_text, :currency,
      :xau_price, :xag_price, :chg_xau, :chg_xag, :pc_xau, :pc_xag, :xau_close, :xag_close,
      :raw_json
    );
    """
    conn.execute(sql, row)
    conn.commit()


def currency_list(csv: str) -> Iterable[str]:
    for c in csv.split(","):
        c = c.strip().upper()
        if c:
            yield c


def main() -> None:
    p = argparse.ArgumentParser(description="Sla live goud/zilver prijzen van goldprice.org op in SQLite.")
    p.add_argument("--db", default="goldprice.sqlite", help="Pad naar SQLite databasebestand")
    p.add_argument("--currencies", default="EUR", help="Komma-gescheiden lijst, bijv. EUR,USD,GBP")
    p.add_argument("--interval", type=int, default=60, help="Polling-interval in seconden (advies: 20-60)")
    p.add_argument("--once", action="store_true", help="Voer 1 fetch uit en stop")
    args = p.parse_args()

    conn = sqlite3.connect(args.db)
    try:
        init_db(conn)

        while True:
            for cur in currency_list(args.currencies):
                url = f"https://data-asg.goldprice.org/dbXRates/{cur}"
                try:
                    payload = get_json(url)
                    insert_tick(conn, cur, payload)
                    item = extract_item(payload) or {}
                    print(f"[{now_utc_iso()}] {cur} xauPrice={item.get('xauPrice')} xagPrice={item.get('xagPrice')}")
                except Exception as e:
                    # Log naar stdout; desgewenst uitbreiden met aparte error-tabel.
                    print(f"[{now_utc_iso()}] ERROR {cur}: {e}")

            if args.once:
                break
            time.sleep(max(1, args.interval))
    finally:
        conn.close()


if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] [--db DB] [--currencies CURRENCIES]
                             [--interval INTERVAL] [--once]
ipykernel_launcher.py: error: unrecognized arguments: --f=/Users/woutertimmer/Library/Jupyter/runtime/kernel-v393e3604adce05023dbc9bb771626664365deaabc.json


SystemExit: 2